## Atividade 03 - Análise de Série Temporal com Python

## Parte 1 - Tratando e arrumando os dados

Essa primeira parte eu fiz para o laboratório, por isso o ambiente virtual utilizado é diferente. Coloquei para mostrar como foi feito o tratamento de dados que resultou em uma planilha única com mais de 400k de linhas de dados sobre movimentação de resíduos sólidos no município de Florianópolis dos anos de 2016 a 2023.

In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import kendalltau
from sklearn.linear_model import TheilSenRegressor

In [ ]:
# Definir o caminho da pasta que contém as planilhas
caminho_pasta = r'C:\Users\CPDI\Documents\GitHub\LARESO_Python\Dados_Entrada'
caminho_saida = r'C:\Users\CPDI\Documents\GitHub\LARESO_Python\Dados_Saida'

# Garantir que a pasta de saída exista
os.makedirs(caminho_saida, exist_ok=True)

# Anos a serem processados
anos = list(range(2016, 2023))

# Processar as planilhas para cada ano
for ano in anos:
    # Carregar a planilha Gerador
    caminho_planilhagerador = os.path.join(caminho_pasta, f'gerador florianópolis {ano}.xlsx')
    planilhagerador = pd.read_excel(caminho_planilhagerador, engine='openpyxl')

    # Carregar a planilha Destinador
    caminho_planilhadestinador = os.path.join(caminho_pasta, f'destinador florianópolis {ano}.xlsx')
    planilhadestinador = pd.read_excel(caminho_planilhadestinador, engine='openpyxl')

    # Excluir as linhas onde "NOME MUNICIPIO GERADOR" for igual a "NOME MUNICIPIO DESTINADOR"
    planilhagerador_sem_duplicatas = planilhagerador[planilhagerador['NOME MUNICIPIO GERADOR'] != planilhagerador['NOME MUNICIPIO DESTINADOR']]

    # Unir a planilha Gerador (sem duplicatas) com a planilha Destinador
    planilha_unida = pd.concat([planilhagerador_sem_duplicatas, planilhadestinador], ignore_index=True)

    # Salvar a planilha sem duplicatas e a união no formato .xlsx
    caminho_saida_planilhagerador = os.path.join(caminho_saida, f'gerador_florianópolis_{ano}_Sem_Duplicatas.xlsx')
    caminho_saida_planilha_unida = os.path.join(caminho_saida, f'Planilha_Unida_{ano}.xlsx')

    # Salvar a planilha Gerador sem duplicatas
    planilhagerador_sem_duplicatas.to_excel(caminho_saida_planilhagerador, index=False, engine='openpyxl')

    # Salvar a planilha unida (Gerador sem duplicatas + Destinador)
    planilha_unida.to_excel(caminho_saida_planilha_unida, index=False, engine='openpyxl')

    print(f'Processamento do ano {ano} concluído. Arquivos salvos: {caminho_saida_planilhagerador} e {caminho_saida_planilha_unida}')

# Definir o caminho da pasta onde as planilhas sem duplicatas e unidas foram salvas
caminho_pasta = r'C:\Users\CPDI\Documents\GitHub\LARESO_Python\Dados_Saida'
caminho_pasta_entrada = r'C:\Users\CPDI\Documents\GitHub\LARESO_Python\Dados_Entrada'

# Anos a serem verificados
anos = list(range(2016, 2023))

# Verificar as planilhas para cada ano
for ano in anos:
    # Carregar a planilha Gerador sem duplicatas
    caminho_planilhagerador_sem_duplicatas = os.path.join(caminho_pasta, f'gerador_florianópolis_{ano}_Sem_Duplicatas.xlsx')
    planilhagerador_sem_duplicatas = pd.read_excel(caminho_planilhagerador_sem_duplicatas, engine='openpyxl')

    # Carregar a planilha Destinador
    caminho_planilhadestinador = os.path.join(caminho_pasta_entrada, f'destinador florianópolis {ano}.xlsx')
    planilhadestinador = pd.read_excel(caminho_planilhadestinador, engine='openpyxl')

    # Carregar a planilha unida
    caminho_planilha_unida = os.path.join(caminho_pasta, f'Planilha_Unida_{ano}.xlsx')
    planilha_unida = pd.read_excel(caminho_planilha_unida, engine='openpyxl')

    # Contar o número de linhas da planilha Gerador (sem duplicatas)
    linhas_gerador_sem_duplicatas = planilhagerador_sem_duplicatas.shape[0]
    
    # Contar o número de linhas da planilha Destinador
    linhas_destinador = planilhadestinador.shape[0]
    
    # Contar o número de linhas da planilha unida
    linhas_unidas = planilha_unida.shape[0]

    # Verificar se o número de linhas unidas é igual à soma das linhas das duas planilhas
    linhas_esperadas = linhas_gerador_sem_duplicatas + linhas_destinador

    # Exibir as informações pra confirmar que foi feito tudo certo
    print(f'Ano {ano}:')
    print(f'Linhas na planilha Gerador (sem duplicatas): {linhas_gerador_sem_duplicatas}')
    print(f'Linhas na planilha Destinador: {linhas_destinador}')
    print(f'Soma esperada: {linhas_esperadas}')
    print(f'Linhas na planilha unida: {linhas_unidas}')

    # Verificar se a união está correta
    if linhas_unidas == linhas_esperadas:
        print(f'União das planilhas de {ano} está correta.\n')
    else:
        print(f'Erro na união das planilhas de {ano}. Linhas unidas: {linhas_unidas}, esperado: {linhas_esperadas}.\n')

In [ ]:
# Lista para armazenar as planilhas unidas de todos os anos
todas_planilhas_unidas = []

# Carregar cada planilha unida de cada ano
for ano in anos:
    # Caminho da planilha unida do ano atual
    caminho_planilha_unida = os.path.join(caminho_saida, f'Planilha_Unida_{ano}.xlsx')
    
    # Verificar se a planilha unida existe
    if os.path.exists(caminho_planilha_unida):
        # Carregar a planilha unida
        planilha_unida = pd.read_excel(caminho_planilha_unida, engine='openpyxl')
        
        # Adicionar a coluna "Ano" para identificar o ano na planilha
        planilha_unida['Ano'] = ano
        
        # Adicionar a planilha à lista
        todas_planilhas_unidas.append(planilha_unida)
        print(f'Planilha unida de {ano} carregada e adicionada à lista.')
    else:
        print(f'Planilha unida de {ano} não encontrada.')

# Concatenar todas as planilhas unidas em uma única planilha
if todas_planilhas_unidas:
    planilha_unica = pd.concat(todas_planilhas_unidas, ignore_index=True)

    # Salvar a planilha única em um arquivo Excel
    caminho_saida_planilha_unica = os.path.join(caminho_saida, 'Planilha_Unica_Todos_Anos.xlsx')
    planilha_unica.to_excel(caminho_saida_planilha_unica, index=False, engine='openpyxl')

    print(f'Planilha única de todos os anos salva em: {caminho_saida_planilha_unica}')
else:
    print('Nenhuma planilha unida foi encontrada para criar a planilha única.')

## Parte 2 - Análise da Série Temporal (2016 - 2023)

Aqui eu mudei o nome da planilha tratada para "dados_atv3" e estou trabalhando no ambiente virtual da disciplina.

In [6]:
# Definir o caminho da pasta que contém a planilha geral 
path = r'C:\Users\CPDI\Documents\GitHub\ENS410064\Dados_Entrada\MTRs_Florianopolis.csv'
path_fig_atv3 = r'C:\Users\CPDI\Documents\GitHub\ENS410064\Dados_Entrada\Figuras_atv3'

data = pd.read_csv(path, sep=';', dtype={9: str, 12: str}) # Coluna 9 e 12 estavam dando problema então converti para string já que são textos

print(data.dtypes)

DATA RECEBIMENTO MANIFESTO                        object
%CODIGO MANIFESTO                                float64
QUANTIDADE GERADOS RESIDUOS (TONELADAS)          float64
QUANTIDADE DESTINADA RESIDUOS (TONELADAS) (1)    float64
TIPO RESIDUO 1                                    object
TIPO RESIDUO 2                                    object
TIPO RESIDUO 3                                    object
CLASSE ITEM_MANIFESTO                             object
TECNOLOGIA DESTINAÇÃO                             object
CNPJ GERADOR                                      object
NOME GERADOR                                      object
NOME MUNICIPIO GERADOR                            object
CNPJ DESTINADOR                                   object
NOME DESTINADOR                                   object
NOME MUNICIPIO DESTINADOR                         object
Ano                                                int64
dtype: object


A minha coluna "DATA RECEBIMENTO MANIFESTO" está no formato "ano-mês-dia 00:00:00", então vou arrumar primeiro.

In [8]:
# Converter a coluna 'DATA RECEBIMENTO MANIFESTO' para o formato datetime, especificando o formato correto
data['DATA RECEBIMENTO MANIFESTO'] = pd.to_datetime(
    data['DATA RECEBIMENTO MANIFESTO'], 
    format='%d/%m/%Y %H:%M', 
    errors='coerce'
)

# Criar novas colunas para o dia, mês e ano, ignorando valores NaT
data['Dia'] = data['DATA RECEBIMENTO MANIFESTO'].dt.day
data['Mês'] = data['DATA RECEBIMENTO MANIFESTO'].dt.month
data['Ano'] = data['DATA RECEBIMENTO MANIFESTO'].dt.year

# Exibir as primeiras linhas para verificar o resultado
print(data[['DATA RECEBIMENTO MANIFESTO', 'Dia', 'Mês', 'Ano']].head())

  DATA RECEBIMENTO MANIFESTO  Dia  Mês     Ano
0                        NaT  NaN  NaN     NaN
1                 2016-01-04  4.0  1.0  2016.0
2                 2016-01-04  4.0  1.0  2016.0
3                 2016-01-04  4.0  1.0  2016.0
4                 2016-01-04  4.0  1.0  2016.0


Análise da série temporal utilizando um lineplot com uma mancha com os valores mínimos e máximos

In [16]:
print(data.dtypes)

DATA RECEBIMENTO MANIFESTO                       datetime64[ns]
%CODIGO MANIFESTO                                       float64
QUANTIDADE GERADOS RESIDUOS (TONELADAS)                 float64
QUANTIDADE DESTINADA RESIDUOS (TONELADAS) (1)           float64
TIPO RESIDUO 1                                           object
TIPO RESIDUO 2                                           object
TIPO RESIDUO 3                                           object
CLASSE ITEM_MANIFESTO                                    object
TECNOLOGIA DESTINAÇÃO                                    object
CNPJ GERADOR                                             object
NOME GERADOR                                             object
NOME MUNICIPIO GERADOR                                   object
CNPJ DESTINADOR                                          object
NOME DESTINADOR                                          object
NOME MUNICIPIO DESTINADOR                                object
Ano                                     

Lineplot

In [33]:
# Função para agregar resíduos por mês e ano, separando gerados e destinados
def agregar_residuos(data):
    # Criar colunas de 'Ano' e 'Mês' a partir da coluna de data
    data['Ano'] = data['DATA RECEBIMENTO MANIFESTO'].dt.year
    data['Mês'] = data['DATA RECEBIMENTO MANIFESTO'].dt.month

    # Agregar resíduos gerados e destinados no mesmo DataFrame
    residuos_mensais = data.groupby(['Ano', 'Mês'])[['QUANTIDADE GERADOS RESIDUOS (TONELADAS)', 'QUANTIDADE DESTINADA RESIDUOS (TONELADAS) (1)']].sum().reset_index()
    residuos_mensais['Data'] = pd.to_datetime(residuos_mensais.rename(columns={'Ano': 'year', 'Mês': 'month'}).assign(day=1)[['year', 'month', 'day']])

    return residuos_mensais

# Função para criar lineplot de resíduos gerados com faixa de mínimos e máximos gerais
def lineplot_gerados(residuos_mensais, localidade, path_fig_atv3):
    min_geral = residuos_mensais['QUANTIDADE GERADOS RESIDUOS (TONELADAS)'].min()
    max_geral = residuos_mensais['QUANTIDADE GERADOS RESIDUOS (TONELADAS)'].max()

    plt.figure(figsize=(12, 6))
    plt.plot(residuos_mensais['Data'], residuos_mensais['QUANTIDADE GERADOS RESIDUOS (TONELADAS)'], label='Gerados', color='blue')
    plt.fill_between(residuos_mensais['Data'], min_geral, max_geral, color='skyblue', alpha=0.3, label='Faixa Min-Max Geral')
    plt.title(f'Série Temporal de Resíduos Gerados (Mensal) - {localidade}')
    plt.xlabel('Anos)')
    plt.ylabel('Quantidade de Resíduos Gerados (Toneladas)')
    plt.legend(loc='upper right')
    plt.grid(axis='y', linestyle='--', color='gray', alpha=0.5)
    plt.xticks(rotation=45, ha='right', fontsize=10)
    plt.tight_layout()

    if not os.path.exists(path_fig_atv3):
        os.makedirs(path_fig_atv3)

    fig_name = os.path.join(path_fig_atv3, f'lineplot_gerados_{localidade}.png')
    plt.savefig(fig_name)
    plt.close()

    print(f"Figura de resíduos gerados salva em: {fig_name}")

# Função para criar lineplot de resíduos destinados com faixa de mínimos e máximos gerais
def lineplot_destinados(residuos_mensais, localidade, path_fig_atv3):
    min_geral = residuos_mensais['QUANTIDADE DESTINADA RESIDUOS (TONELADAS) (1)'].min()
    max_geral = residuos_mensais['QUANTIDADE DESTINADA RESIDUOS (TONELADAS) (1)'].max()

    plt.figure(figsize=(12, 6))
    plt.plot(residuos_mensais['Data'], residuos_mensais['QUANTIDADE DESTINADA RESIDUOS (TONELADAS) (1)'], label='Destinados', color='green')
    plt.fill_between(residuos_mensais['Data'], min_geral, max_geral, color='lightgreen', alpha=0.3, label='Faixa Min-Max Geral')
    plt.title(f'Série Temporal de Resíduos Destinados (Mensal) - {localidade}')
    plt.xlabel('Anos')
    plt.ylabel('Quantidade de Resíduos Destinados (Toneladas)')
    plt.legend(loc='upper right')
    plt.grid(axis='y', linestyle='--', color='gray', alpha=0.5)
    plt.xticks(rotation=45, ha='right', fontsize=10)
    plt.tight_layout()

    fig_name = os.path.join(path_fig_atv3, f'lineplot_destinados_{localidade}.png')
    plt.savefig(fig_name)
    plt.close()

    print(f"Figura de resíduos destinados salva em: {fig_name}")

# Função para criar lineplot de resíduos gerados e destinados juntos (sem faixa de mínimos e máximos)
def lineplot_gerados_destinados(residuos_mensais, localidade, path_fig_atv3):
    plt.figure(figsize=(12, 6))
    plt.plot(residuos_mensais['Data'], residuos_mensais['QUANTIDADE GERADOS RESIDUOS (TONELADAS)'], label='Gerados', color='blue')
    plt.plot(residuos_mensais['Data'], residuos_mensais['QUANTIDADE DESTINADA RESIDUOS (TONELADAS) (1)'], label='Destinados', color='green')
    plt.title(f'Série Temporal de Resíduos Gerados e Destinados (Mensal) - {localidade}')
    plt.xlabel('Anos')
    plt.ylabel('Quantidade de Resíduos (Toneladas)')
    plt.legend(loc='upper right')
    plt.grid(axis='y', linestyle='--', color='gray', alpha=0.5)
    plt.xticks(rotation=45, ha='right', fontsize=10)
    plt.tight_layout()

    fig_name = os.path.join(path_fig_atv3, f'lineplot_gerados_destinados_{localidade}.png')
    plt.savefig(fig_name)
    plt.close()

    print(f"Figura de resíduos gerados e destinados salva em: {fig_name}")

# Executar a análise para Florianópolis
analise_completa(data, 'Florianópolis', path_fig_atv3)

Figura de resíduos gerados salva em: C:\Users\CPDI\Documents\GitHub\ENS410064\Dados_Entrada\Figuras_atv3\lineplot_gerados_Florianópolis.png
Figura de resíduos destinados salva em: C:\Users\CPDI\Documents\GitHub\ENS410064\Dados_Entrada\Figuras_atv3\lineplot_destinados_Florianópolis.png
Figura de resíduos gerados e destinados salva em: C:\Users\CPDI\Documents\GitHub\ENS410064\Dados_Entrada\Figuras_atv3\lineplot_gerados_destinados_Florianópolis.png


Boxplot

In [41]:
# Função para agregar resíduos gerados por mês e ano
def agregar_residuos_gerados(data):
    data['Ano'] = data['DATA RECEBIMENTO MANIFESTO'].dt.year
    data['Mês'] = data['DATA RECEBIMENTO MANIFESTO'].dt.month
    residuos_mensais_gerados = data.groupby(['Ano', 'Mês'])['QUANTIDADE GERADOS RESIDUOS (TONELADAS)'].sum().reset_index()
    return residuos_mensais_gerados

# Função para criar boxplot de resíduos gerados por mês
def boxplot_residuos_gerados_mensal(residuos_mensais_gerados, localidade, path_fig_atv3):
    # Criar um boxplot para resíduos gerados por mês
    plt.figure(figsize=(12, 6))
    residuos_mensais_gerados['Mês'] = residuos_mensais_gerados['Mês'].apply(lambda x: pd.to_datetime(x, format='%m').strftime('%b'))

    # Criar o boxplot usando 'tick_labels'
    plt.boxplot(
        [residuos_mensais_gerados[residuos_mensais_gerados['Mês'] == mes]['QUANTIDADE GERADOS RESIDUOS (TONELADAS)'] 
         for mes in residuos_mensais_gerados['Mês'].unique()],
        tick_labels=residuos_mensais_gerados['Mês'].unique()
    )

    plt.title(f'Boxplot Mensal de Resíduos Gerados - {localidade}')
    plt.xlabel('Mês')
    plt.ylabel('Quantidade de Resíduos Gerados (Toneladas)')
    plt.tight_layout()
    plt.grid(axis='y', linestyle='--', color='gray', alpha=0.1)

    # Salvar figura
    fig_name = os.path.join(path_fig_atv3, f'boxplot_residuos_gerados_{localidade}.png')
    plt.savefig(fig_name)
    plt.close()

    print(f"Boxplot de resíduos gerados por mês salvo em: {fig_name}")

# Função principal para a análise completa
def analise_boxplot_residuos_gerados(data, localidade, path_fig_atv3):
    residuos_mensais_gerados = agregar_residuos_gerados(data)
    boxplot_residuos_gerados_mensal(residuos_mensais_gerados, localidade, path_fig_atv3)

# Executar a análise para Florianópolis
analise_boxplot_residuos_gerados(data, 'Florianópolis', path_fig_atv3)

Boxplot de resíduos gerados por mês salvo em: C:\Users\CPDI\Documents\GitHub\ENS410064\Dados_Entrada\Figuras_atv3\boxplot_residuos_gerados_Florianópolis.png


Estatísticas univariadas

In [45]:
# Função para calcular estatísticas univariadas usando o método describe()
def estatisticas_unificadas(data):
    resultados = {}

    # Estatísticas por dia da semana
    data['Dia da Semana'] = data['DATA RECEBIMENTO MANIFESTO'].dt.day_name()
    estatisticas_dia_semana = data.groupby('Dia da Semana')['QUANTIDADE GERADOS RESIDUOS (TONELADAS)'].describe().reset_index()
    resultados['dia_semana'] = estatisticas_dia_semana

    # Estatísticas por mês do ano
    data['Mês'] = data['DATA RECEBIMENTO MANIFESTO'].dt.month
    estatisticas_mes = data.groupby('Mês')['QUANTIDADE GERADOS RESIDUOS (TONELADAS)'].describe().reset_index()
    resultados['mes'] = estatisticas_mes

    # Estatísticas por ano (2016 a 2023)
    data['Ano'] = data['DATA RECEBIMENTO MANIFESTO'].dt.year
    estatisticas_ano = data[(data['Ano'] >= 2016) & (data['Ano'] <= 2023)].groupby('Ano')['QUANTIDADE GERADOS RESIDUOS (TONELADAS)'].describe().reset_index()
    resultados['ano'] = estatisticas_ano

    return resultados

# Função para executar a análise e salvar em .csv de forma separada
def analise_estatisticas_unificadas(data, path_fig_atv3):
    estatisticas_completas = estatisticas_unificadas(data)

    # Salvar os arquivos .csv em UTF-8 com BOM
    csv_dia_semana = os.path.join(path_fig_atv3, 'estatisticas_dia_semana.csv')
    csv_mes = os.path.join(path_fig_atv3, 'estatisticas_mes.csv')
    csv_ano = os.path.join(path_fig_atv3, 'estatisticas_ano.csv')

    estatisticas_completas['dia_semana'].to_csv(csv_dia_semana, index=False, encoding='utf-8-sig')
    estatisticas_completas['mes'].to_csv(csv_mes, index=False, encoding='utf-8-sig')
    estatisticas_completas['ano'].to_csv(csv_ano, index=False, encoding='utf-8-sig')

    print(f"Estatísticas por dia da semana salvas em: {csv_dia_semana}")
    print(f"Estatísticas por mês salvas em: {csv_mes}")
    print(f"Estatísticas por ano salvas em: {csv_ano}")

# Executar a análise para Florianópolis
analise_estatisticas_unificadas(data, path_fig_atv3)

Estatísticas por dia da semana salvas em: C:\Users\CPDI\Documents\GitHub\ENS410064\Dados_Entrada\Figuras_atv3\estatisticas_dia_semana.csv
Estatísticas por mês salvas em: C:\Users\CPDI\Documents\GitHub\ENS410064\Dados_Entrada\Figuras_atv3\estatisticas_mes.csv
Estatísticas por ano salvas em: C:\Users\CPDI\Documents\GitHub\ENS410064\Dados_Entrada\Figuras_atv3\estatisticas_ano.csv


Sazonalidade

In [28]:
# Função para agregar resíduos por mês e ano
def agregar_residuos_gerados(data):
    data['Ano'] = data['DATA RECEBIMENTO MANIFESTO'].dt.year
    data['Mês'] = data['DATA RECEBIMENTO MANIFESTO'].dt.month
    residuos_mensais_gerados = data.groupby(['Ano', 'Mês'])['QUANTIDADE GERADOS RESIDUOS (TONELADAS)'].sum().reset_index()

    # Criar coluna de data para série temporal
    residuos_mensais_gerados = residuos_mensais_gerados.rename(columns={'Ano': 'year', 'Mês': 'month'})
    residuos_mensais_gerados['Data'] = pd.to_datetime(residuos_mensais_gerados.assign(day=1)[['year', 'month', 'day']])

    return residuos_mensais_gerados

# Função para calcular o estimador de Thiel-Sen
def thiel_sen_estimador(series):
    X = np.arange(len(series)).reshape(-1, 1)
    y = series.values
    model = TheilSenRegressor().fit(X, y)
    slope = model.coef_[0]
    intercept = model.intercept_
    return slope, intercept

# Função para analisar tendência com o Estimador de Thiel-Sen
def analise_tendencia_thiel_sen(data, path_fig_atv3):
    residuos_mensais_gerados = agregar_residuos_gerados(data)

    # Série temporal de resíduos gerados
    series = residuos_mensais_gerados.set_index('Data')['QUANTIDADE GERADOS RESIDUOS (TONELADAS)']

    # Estimador de Thiel-Sen
    slope, intercept = thiel_sen_estimador(series)
    print(f"Estimador de Thiel-Sen: Inclinação={slope:.4f} ton/mês, Intercepto={intercept:.4f}")

    # Criar uma análise para incluir na figura
    analise = (
        f"$\\bf{{Inclinação\\ do\\ Estimador\\ de\\ Thiel\\text{{-}}Sen:}}$ {slope:.4f} ton/mês.\n"
        "Isso indica que, em média, os resíduos gerados estão aumentando\n"
        "94,54 toneladas a cada mês, no período de 2016 a 2023."
    )

    # Plotar série temporal com tendência
    plt.figure(figsize=(12, 8))
    plt.plot(series.index, series, label='Resíduos Gerados', color='blue')

    # Adicionar a linha de tendência (Thiel-Sen)
    tendencia = intercept + slope * np.arange(len(series))
    plt.plot(series.index, tendencia, label='Tendência (Thiel-Sen)', color='red', linestyle='--')

    # Adicionar a explicação no canto superior esquerdo em um único box
    plt.text(0.02, 0.95, analise, transform=plt.gca().transAxes, fontsize=10,
             bbox=dict(facecolor='white', alpha=0.8), verticalalignment='top',
             horizontalalignment='left')

    plt.title('Análise de Tendência - Estimador de Thiel-Sen')
    plt.xlabel('Anos')
    plt.ylabel('Quantidade de Resíduos Gerados (Toneladas)')
    plt.legend(loc='upper right')
    plt.grid(axis='y', linestyle='--', color='gray', alpha=0.5)
    plt.xticks(rotation=45, ha='right', fontsize=10)

    # Salvar figura
    fig_name = os.path.join(path_fig_atv3, 'tendencia_thiel_sen_residuos_gerados.png')
    plt.tight_layout()
    plt.savefig(fig_name)
    plt.close()

    print(f"Figura de análise de tendência (Thiel-Sen) salva em: {fig_name}")

# Caminho para salvar as figuras
path_fig_atv3 = r'C:\Users\CPDI\Documents\GitHub\ENS410064\Dados_Entrada\Figuras_atv3'

# Executar a análise
analise_tendencia_thiel_sen(data, path_fig_atv3)

Estimador de Thiel-Sen: Inclinação=94.5420 ton/mês, Intercepto=846.0623
Figura de análise de tendência (Thiel-Sen) salva em: C:\Users\CPDI\Documents\GitHub\ENS410064\Dados_Entrada\Figuras_atv3\tendencia_thiel_sen_residuos_gerados.png
